# Gemini Experiments


---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installation

In [ ]:
!pip install git+https://github.com/shahadMAlshalawi/Modular-Arabic-VQA.git --no-warn-conflicts --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import textwrap
import tqdm
from PIL import Image
import requests
import torch
import numpy as np
import pandas as pd
from google.colab import userdata
import evaluate
import json
import datasets
import aravqa

### Configuration

In [ ]:
from aravqa.core import Config, CaptionSelection
import textwrap
from google.colab import userdata
import torch

# Initialize General Constants
OPENAI_API_KEY = userdata.get('GPT_API_KEY')

# Initialize the configuration object
config = Config()

# -------------------- Dataset Configuration --------------------
config.VDS_PATH = "ShahadMAlshalawi/OKVQA-Violet-Captions-with-bert-similarity-Final"  # The path of Violet 6 captions for all the OKVQA dataset
config.BDS_PATH = "ShahadMAlshalawi/OKVQA-BiT-Captions-with-bert-similarity-Final"  # The path of BiT 6 captions for all the OKVQA dataset
config.GPT4oDS_PATH = "ShahadMAlshalawi/OKVQA-GPT-4o-Captions-with-bert-similarity-Final"  # Path for GPT4o 6 captions for all the OKVQA dataset

config.LANGUAGE = "ar"  # Target language for questions and answers
config.SPLIT = "validation"  # Dataset split (e.g., train, validation, test)
config.USERNAME = "ShahadMAlshalawi"  # HF User's name or identifier

# -------------------- Device Configuration --------------------
config.DEVICE = "cpu"  # Execution device (e.g., 'cpu' or 'cuda') since here we request the gemini responces we do not need cuda

# -------------------- Processing Settings --------------------
config.BATCH_SIZE = 5  # Batch size for data processing
config.CAPTIONS = ["bit","violet", "GPT4o"]  # Captioning models to include (e.g. "bit","violet","GPT4o","None"). we use the short name "bit" for the Bidirectional transformer model (AraBERT32-Flickr8k)
config.CAPTIONS_SEPARATOR = "\n"  # Default separator between captions (e.g., new line)
config.NUM_CAPTIONS = 6  # Number of captions to select (-1 for all captions)
config.CAPTION_SELECTION = CaptionSelection.LOW_SIMILARITY_QUESTION
config.RANDOM_SEED = 42 # Random seed for reproducibility

config.PATH_RESULT_FILE = f"drive/MyDrive/ColabData/Modular_Arabic_VQA_System_Experiments_Results/OKVQA-ar_experiments_results/gemini/Final_Results/{config.CAPTION_SELECTION}-{'-'.join(config.CAPTIONS)}.csv"

# -------------------- API and Model Configuration --------------------
config.API_KEY = userdata.get('GEMINI_API_KEY')  # API key for external services
config.MODEL_NAME = "models/gemini-1.5-flash"  # Model name for text generation

# -------------------- Text Generation Settings --------------------
config.GENERATION_CONFIG = {
    "temperature": 0.0,  # Controls randomness in text generation
    "top_p": 0.95,  # Nucleus sampling threshold
    "top_k": 40,  # Limits sampling to the top-k tokens
    "max_output_tokens": 20,  # Maximum number of tokens in the output (the answers need to be short)
    "response_mime_type": "text/plain",  # Format of the generated text
}
# -------------------- Few-Shot Learning Configuration ---------------------------

# -------------------- Prompt and Instruction Settings -------------------

# Optional System Instruction: General guideline for the LLM. [str,None]
config.SYSTEM_INSTRUCTION = textwrap.dedent(
    """
    You are a highly capable language model specialized in answering questions based on provided image captions.
    Here are five examples of Question-Captions pairs to guide you:
    Example 1:
    Captions:{"طفل يحمل مضرب بيسبول في الحديقة", "الصورة بالأبيض والأسود", "الطفل يبدو في وضع الاستعداد لضرب الكرة", "الشمس تضيء المنطقة المحيطة", "السياج الخشبي يظهر في الخلفية", "شجرة كبيرة تضيف ظلاً للمشهد"}
    Question:"ما هو لون العشب في هذه الصورة؟"
    Answer : "أبيض وأسود"

    Example 2:
    Captions:{"غرفة نوم بمكتب دراسي في الزاوية", "كرسي مكتب جلد مريح أمام الطاولة", "سرير بجانب المكتب مغطى ببطانية", "صور مؤطرة معلقة على الجدار", "نافذة كبيرة توفر إضاءة طبيعية" ,"طابعة وأوراق مرتبة على رف جانبي"}
    Question:{مما يتكون الكرسي}
    Answer : {الجلد والمعادن والخشب والبلاستك}

    Example 3:
    Captions:{"مبنى من الطوب بلون أحادي مع نافذة خشبية." ,"لافتة شارع معلقة مكتوب عليها \"ROMA 201\".", "باب خشبي كبير في منتصف المبنى." ,"عمود إنارة يقف بجانب رصيف الشارع." ,"صورة بالأبيض والأسود لشارع في مدينة أوروبية." ,"لافتة منطقة محظورة تشير إلى قيود المرور."}
    Question:{هل هذه الصورة ملونة}
    Answer : {لا}

    Example 4:
    Captions:{"لافتة زرقاء مكتوب عليها \"SALISBURY RD\".", "لافتة صفراء متهالكة مكتوب عليها \"LANDS LANE\" وتحمل اسم \"CITY OF SYDNEY\".", "لافتة خضراء مكتوب عليها \"CRUIKSHANK ST\".", "لافتة بيضاء عليها \"ROSS ST\" مكتوبًا بخط كبير وواضح.", "اللافتة بها رمز لجمعية تنظيف في اليسار.", "الاسم \"CAMPERDOWN\" مذكور فوق اسم الشارع."}
    Question:{كم عدد اللافتات؟}
    Answer : {أربع}

    Example 5:
    Captions:{"مبنى ضخم مكون من طوب بني قاتم." ,"برجان متساويان في الارتفاع على جانبي المبنى." ,"ساعة كبيرة ظاهرة على أحد الأبراج." ,"أشجار خضراء بجوار المبنى تضيف جمالية للمشهد." ,"ساحة كبيرة أمام المبنى حيث يسير الناس." ,"سماء زرقاء صافية في الخلفية تعكس يومًا مشمسًا."}
    Question:{كم عدد ناطحات السحاب هناك؟}.
    Answer : {اثنتان}
    Now, for each new pair your task is to analyze the captions and generate accurate, concise answers in the same language as the question.
    Ensure your response is relevant, clear, and avoids unnecessary details.
    """
).strip()

# Prompt Template: Structured format for generating questions and answers
config.PROMPT_TEMPLATE = textwrap.dedent(
    """
    Analyze the following image captions and answer the given question in the same language:
    Captions:{context}
    Question:{question}
    Answer concisely:
    """
).strip()



print(f"System Instruction:\n{config.SYSTEM_INSTRUCTION}")
print(f"\nPrompt Template:\n{config.PROMPT_TEMPLATE}")

System Instruction:
You are a highly capable language model specialized in answering questions based on provided image captions.
Here are five examples of Question-Captions pairs to guide you:
Example 1:
Captions:{"طفل يحمل مضرب بيسبول في الحديقة", "الصورة بالأبيض والأسود", "الطفل يبدو في وضع الاستعداد لضرب الكرة", "الشمس تضيء المنطقة المحيطة", "السياج الخشبي يظهر في الخلفية", "شجرة كبيرة تضيف ظلاً للمشهد"}
Question:"ما هو لون العشب في هذه الصورة؟"
Answer : "أبيض وأسود"

Example 2:
Captions:{"غرفة نوم بمكتب دراسي في الزاوية", "كرسي مكتب جلد مريح أمام الطاولة", "سرير بجانب المكتب مغطى ببطانية", "صور مؤطرة معلقة على الجدار", "نافذة كبيرة توفر إضاءة طبيعية" ,"طابعة وأوراق مرتبة على رف جانبي"}
Question:{مما يتكون الكرسي}
Answer : {الجلد والمعادن والخشب والبلاستك}

Example 3:
Captions:{"مبنى من الطوب بلون أحادي مع نافذة خشبية." ,"لافتة شارع معلقة مكتوب عليها "ROMA 201".", "باب خشبي كبير في منتصف المبنى." ,"عمود إنارة يقف بجانب رصيف الشارع." ,"صورة بالأبيض والأسود لشارع في مدينة أوروبية." ,"

### Login Hugging Face

In [ ]:
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

### DataSet & DataLoader

In [ ]:
from datasets import load_dataset
from aravqa.datasets.utils import prepare_dataset
from aravqa.datasets.utils import compute_similarity_captions
from aravqa.datasets import OKVQADataset
from aravqa.datasets import OKVQADataLoader
from aravqa.datasets import VQAv2Dataset
from aravqa.datasets import VQAv2DataLoader

In [ ]:
# Load Datasets

BDS = load_dataset(config.BDS_PATH,split=Config.SPLIT).remove_columns("features")
VDS= load_dataset(config.VDS_PATH,split=Config.SPLIT).remove_columns("features")
GPT4oDS = load_dataset(config.GPT4oDS_PATH,split=Config.SPLIT)


README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

validation-00000-of-00005.parquet:   0%|          | 0.00/297M [00:00<?, ?B/s]

validation-00001-of-00005.parquet:   0%|          | 0.00/306M [00:00<?, ?B/s]

validation-00002-of-00005.parquet:   0%|          | 0.00/302M [00:00<?, ?B/s]

validation-00003-of-00005.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/335M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/5046 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/942 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/263M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/5046 [00:00<?, ? examples/s]

####**Compute Similarity Captions (Bert-score)**
* This section can be executed once to compute the similatity Score (bertScore) between captions and corresponding question and answers

* After saving the similarity scores no need to run it again durring expermints.

In [ ]:
'''
!pip install evaluate
!pip install bert_score
'''

In [ ]:
'''
from typing import List, Dict, Callable, Optional
from tqdm import tqdm
import evaluate
import numpy as np

def compute_bertscore(predictions: List[str], references: List[List[str]]) -> float:
    """
    Computes the BERTScore for a set of predictions and references.

    Args:
        predictions (List[str]): List of predicted sentences.
        references (List[List[str]]): List of lists of reference sentences.

    Returns:
        float: The computed BERTScore. Returns float("-inf") if computation fails.

    Raises:
        ValueError: If predictions or references are empty, or if their lengths do not match.
    """
    if not predictions or not references:
        raise ValueError("Predictions and references must not be empty.")
    if len(predictions) != len(references):
        raise ValueError("The number of predictions must match the number of reference sets.")

    bertscore_scorer = evaluate.load("bertscore")
    try:
        result = bertscore_scorer.compute(predictions=predictions, references=references, lang="ar", model_type="distilbert-base-multilingual-cased")
        return result["f1"]
    except Exception as e:
        print(f"Error computing bertscore: {e}")
        return float("-inf")  # Indicate computation failure

# ..................................................
# Test

predictions = ["دراجة نارية بيضاء وسوداء متوقفة في موقف للسيارات"]
references = [
            [
              "سباق",
              "يركب",
              "موتوكروس",
              "دراجة نارية",
            ],

             ]

bert_score = compute_bertscore(predictions=predictions,
                                references=references
                                )
print(f"Bert Score: {bert_score}")
'''

In [ ]:
'''
from aravqa.datasets.utils import compute_similarity_captions


VDS = compute_similarity_captions(VDS,
                                  question_similarity_scorer = compute_bertscore,
                                  answer_similarity_scorer = compute_bertscore
                                  )

print(f"VDS captions features: {VDS.features['captions']}")
'''

In [ ]:
#VDS.push_to_hub(f"{config.USERNAME}/OKVQA-Violet-Captions-with-bert-similarity-Final")


In [ ]:
'''
from aravqa.datasets.utils import compute_similarity_captions


BDS = compute_similarity_captions(BDS,
                                  question_similarity_scorer = compute_bertscore,
                                  answer_similarity_scorer = compute_bertscore
                                  )

print(f"BDS captions features: {BDS.features['captions']}")
'''

In [ ]:
#BDS.push_to_hub(f"{config.USERNAME}/OKVQA-BiT-Captions-with-bert-similarity-Final")

In [ ]:
'''
from aravqa.datasets.utils import compute_similarity_captions

GPT4oDS = compute_similarity_captions(GPT4oDS,
                                  question_similarity_scorer = compute_bertscore,
                                  answer_similarity_scorer = compute_bertscore
                                  )

print(f"GPT4oDS captions features: {GPT4oDS.features['captions']}")
'''

In [ ]:
#GPT4oDS.push_to_hub(f"{config.USERNAME}/OKVQA-GPT-4o-Captions-with-bert-similarity-Final")


#### **DataLoader (Prompts + Batches Preperation)**

In [ ]:
# Dataset
dataset = OKVQADataset(BDS,VDS, GPT4oDS)
print(f"Dataset length: {len(dataset)}")
print(f"Dataset sample keys: {dataset[0].keys()}")
print(f"Dataset sample metadata: {dataset[0]['metadata']}")
print(f"Dataset sample question: {dataset[0]['question']}")
print(f"Dataset sample answers: {dataset[0]['answers']}")
print(f"Dataset sample bit captions: {dataset[0]['bit']}")
print(f"Dataset sample violet captions: {dataset[0]['violet']}")
print(f"Dataset sample GPT4o captions: {dataset[0]['GPT4o']}")

Dataset length: 5046
Dataset sample keys: dict_keys(['metadata', 'image', 'question', 'answers', 'bit', 'violet', 'GPT4o'])
Dataset sample metadata: {'image_id': 297147, 'question_id': 2971475, 'question_type': 'one', 'answer_type': 'other', 'confidence': 3}
Dataset sample question: في أي رياضة يمكنك استخدام هذا؟
Dataset sample answers: [{'answer': 'سباق', 'confidence': 'yes', 'id': 1}, {'answer': 'سباق', 'confidence': 'yes', 'id': 2}, {'answer': 'سباق', 'confidence': 'yes', 'id': 3}, {'answer': 'سباق', 'confidence': 'yes', 'id': 4}, {'answer': 'سباق', 'confidence': 'yes', 'id': 5}, {'answer': 'سباق', 'confidence': 'yes', 'id': 6}, {'answer': 'موتوكروس', 'confidence': 'yes', 'id': 7}, {'answer': 'موتوكروس', 'confidence': 'yes', 'id': 8}, {'answer': 'يركب', 'confidence': 'yes', 'id': 9}, {'answer': 'يركب', 'confidence': 'yes', 'id': 10}]
Dataset sample bit captions: [{'caption': 'امراة ترتدي بدلة سوداء وبيضاء تركب دراجتها في الشارع', 'id': 0, 'similarity_answers': [0.8597190380096436], 

In [ ]:
# DataLoader
dataloader = OKVQADataLoader(dataset, config).get_dataloader()
batch = next(iter(dataloader))
print(f"Batch keys: {batch.keys()}")
print(f"\nBatch prompts:\n")
print("\n\n".join(batch['prompts'][:3]))

Batch keys: dict_keys(['question_id', 'question', 'image_id', 'prompts', 'answers'])

Batch prompts:

Analyze the following image captions and answer the given question in the same language:
Captions:
مقعد الدراجة مريح ومصمم لجلسة طويلة
الدراجة من طراز هوندا معروفة بتصميمها الأنيق
الشعار الواضح للدراجة يضيف لمسة فخامة
العجلات مصممة للثبات والسرعة
امراة ترتدي بدلة سوداء وبيضاء تركب دراجة ثلاثية العجلات
امراة ترتدي بدلة سوداء وبيضاء تركب دراجة بخارية
Question:في أي رياضة يمكنك استخدام هذا؟
Answer concisely:

Analyze the following image captions and answer the given question in the same language:
Captions:
باب مفتوح يؤدي إلى غرفة بمنطقة جلوس مريحة.
سقف خشبي مضاء بضوء طبيعي من فتحة علوية.
امراة تجلس على طاولة في فناء خلفي
امراة تجلس على طاولة خضراء في الهواء الطلق
امراة تجلس على طاولة في مطعم
حمام عصري بتصميم أنيق وجدران رمادية.
Question:اذكر نوع النبات هذا؟
Answer concisely:

Analyze the following image captions and answer the given question in the same language:
Captions:
شخصان يجلسان في

### Question Answering

In [ ]:
import pandas as pd
from aravqa.modules.question_answering import GeminiAnswerer

llm = GeminiAnswerer(config)
print(llm)

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={'temperature': 0.0, 'top_p': 0.95, 'top_k': 40, 'max_output_tokens': 20, 'response_mime_type': 'text/plain'},
    safety_settings={},
    tools=None,
    system_instruction='You are a highly capable language model specialized in answering questions based on provided image captions.\nHere are five examples of Question-Captions pairs to guide you:\nExample 1:\nCaptions:{"طفل يحمل مضرب بيسبول في الحديقة", "الصورة بالأبيض والأسود", "الطفل يبدو في وضع الاستعداد لضرب الكرة", "الشمس تضيء المنطقة المحيطة", "السياج الخشبي يظهر في الخلفية", "شجرة كبيرة تضيف ظلاً للمشهد"}\nQuestion:"ما هو لون العشب في هذه الصورة؟"\nAnswer : "أبيض وأسود"\n\nExample 2:\nCaptions:{"غرفة نوم بمكتب دراسي في الزاوية", "كرسي مكتب جلد مريح أمام الطاولة", "سرير بجانب المكتب مغطى ببطانية", "صور مؤطرة معلقة على الجدار", "نافذة كبيرة توفر إضاءة طبيعية" ,"طابعة وأوراق مرتبة على رف جانبي"}\nQuestion:{مما يتكون الكرسي}\nAnswer : {الجلد والمع

In [ ]:
outputs = llm.generate_from_dataloader(dataloader)
pd.DataFrame.from_dict(outputs)

Generating predictions from dataloader: 100%|██████████| 1010/1010 [1:43:57<00:00,  6.18s/it]


,question_id,questions,image_id,answers,predictions
0,2971475,في أي رياضة يمكنك استخدام هذا؟,297147,"[سباق, سباق, سباق, سباق, سباق, سباق, موتوكروس,...",لا شيء.
1,3397615,اذكر نوع النبات هذا؟,339761,"[كرمة, كرمة, كرمة, كرمة, تسلق, تسلق, تبدو وكأن...",لا يمكن تحديد نوع النبات من الوصف المقدم.
2,3575865,أي لعبة هذه؟,357586,"[حيوان محشو, حيوان محشو, حيوان محشو, حيوان محش...",لعبة الأدغال.
3,949225,أي جزء من هذا الحيوان يمكن استخدامه إذا كان يل...,94922,"[فم, فم, فم, فم, فم, فم, فم, فم, فم, فم]",فمه.
4,2076115,ماذا يمكن أن يحمل هذا الرجل في تلك الحقيبة الح...,207611,"[قماش, قماش, قماش, قماش, طعام, طعام, غداء, غدا...",مستندات، محفظة، هاتف، مفاتيح.
...,...,...,...,...,...
5041,101235,ما هي قطعة المعدات الرياضية الموجودة في يد الر...,10123,"[مضرب تنس, مضرب تنس, مضرب تنس, مضرب تنس, مضرب ...",مضرب تنس
5042,4917845,نمط هذا القميص الأخضر يسمى ماذا؟,491784,"[منقوشة, منقوشة, منقوشة, منقوشة, منقوشة, منقوش...",لا يمكن تحديد نمط القميص من الوصف.
5043,3629415,أين تجد هذا الحيوان في البرية؟,362941,"[أفريقيا, أفريقيا, أفريقيا, أفريقيا, أفريقيا, ...",أفريقيا
5044,1105875,هل يمكنك تخمين نوع المادة المستخدمة لصنع هذه ا...,110587,"[صوف, صوف, صوف, صوف, صوف, صوف, غزل, غزل, البول...",لا يمكن تحديد نوع مادة القبعة من الوصف.


### Evaluation

In [ ]:
from aravqa.modules.evaluation import BLEUEvaluator
from aravqa.modules.evaluation import BERTScoreEvaluator
from aravqa.modules.evaluation import FuzzEvaluator

# ..........................................
bleu_evaluator = BLEUEvaluator(max_order=1)
bleu_results = bleu_evaluator.evaluate(predictions=outputs['predictions'],
                                       references=outputs['answers']
                                       )

bleu_results['overall_bleu'] = [bleu_results['overall_bleu']]
bleu_results['overall_bleu'].extend([None]*(len(bleu_results['bleu'])-1))


# ..........................................
BertScore_evaluator = BERTScoreEvaluator()
bertScore_results = BertScore_evaluator.evaluate(predictions=outputs['predictions'],
                                               references=outputs['answers']
                                               )

bertScore_results['overall_f1_bertscore'] = [bertScore_results['overall_f1_bertscore']]
bertScore_results['overall_f1_bertscore'].extend([None]*(len(bertScore_results['f1_bertscore'])-1))

# ..........................................
fuzzy_evaluator = FuzzEvaluator(OPENAI_API_KEY)
fuzzy_results = fuzzy_evaluator.evaluate(predictions=outputs['predictions'],
                                       references=outputs['answers'],
                                         questions=outputs['questions']
                                       )

fuzzy_results['fuzz_overall_accuracy'] = [fuzzy_results['fuzz_overall_accuracy']]
fuzzy_results['fuzz_overall_accuracy'].extend([None]*(len(fuzzy_results['fuzz_accuracy'])-1))

# ..........................................
outputs['bleu'] = bleu_results['bleu']
outputs['f1_bertscore'] = bertScore_results['f1_bertscore']
outputs['fuzz_accuracy'] = fuzzy_results['fuzz_accuracy']
outputs['overall_bleu'] = bleu_results['overall_bleu']
outputs['overall_f1_bertscore'] = bertScore_results['overall_f1_bertscore']
outputs['fuzz_overall_accuracy'] = fuzzy_results['fuzz_overall_accuracy']


outputs_df = pd.DataFrame.from_dict(outputs)

In [ ]:
# save to csv file
outputs_df.to_csv(config.PATH_RESULT_FILE,index=False)

In [ ]:
outputs_df

In [ ]:
print(config.PATH_RESULT_FILE)

drive/MyDrive/ColabData/Modular_Arabic_VQA_System_Experiments_Results/OKVQA-ar_experiments_results/gemini/Final_Results/low_similarity_question-bit-violet-GPT4o.csv


## Showing Results